# 데이터 처리

- 데이터 타입 변환(물량, 금액을 숫자형으로)
- 결측치 포함된 행 제거
- 품종 컬럼 제거
- 일자에서 연도, 월, 일 추출하여 별도 컬럼으로 추가
- 계절 컬럼 추가
- [일자, 부류, 품목, 도매시장] 기준으로 데이터 그룹화
- 총물량(kg), 평균물량(kg), 총금액(원), 평균금액(원), 총단가(원/kg), 평균단가(원/kg) 추가
- 총단가(원/kg): 총금액(원) ÷ 총물량(kg)
- 평균단가(원/kg): 평균금액(원) ÷ 평균물량(kg)
- 음수 값 제거
- 컬럼 순서 재정렬

In [1]:
import pandas as pd

df_agromarket = pd.read_csv('../data/db/preprocessed_agromarket.csv')
df_agromarket.head(5)

,일자,부류,품목,품종,도매시장,물량(kg),금액(원)
0,20190201,버섯류,새송이,새송이버섯(일반),강릉,1303.0,4081500.0
1,20190201,조미채소류,마늘,마늘쫑(수입),강릉,224.0,586000.0
2,20190201,조미채소류,마늘,깐마늘,강릉,1287.0,7259700.0
3,20190201,조미채소류,마늘,풋마늘,강릉,210.0,491000.0
4,20190201,조미채소류,풋고추,아삭이,강릉,62.0,244000.0


In [2]:
# 결측치, 음수값, 중복값 확인
missing_values = df_agromarket.isnull().sum()

# 숫자형 데이터만 선택
numeric_columns = df_agromarket.select_dtypes(include=['number'])
negative_values = (numeric_columns < 0).sum()

duplicate_values = df_agromarket.duplicated().sum()

In [3]:
missing_values

일자         0
부류         0
품목         0
품종        38
도매시장       0
물량(kg)     0
금액(원)      0
dtype: int64

In [4]:
negative_values

일자        0
물량(kg)    0
금액(원)     0
dtype: int64

In [5]:
duplicate_values

np.int64(7250)

In [6]:
# 컬럼명을 영어로 변경
df_agromarket.columns = [
    'date', 'category', 'item', 'variety', 'wholesale_market',
    'quantity_kg', 'price_won'
]

df_agromarket.head()

,date,category,item,variety,wholesale_market,quantity_kg,price_won
0,20190201,버섯류,새송이,새송이버섯(일반),강릉,1303.0,4081500.0
1,20190201,조미채소류,마늘,마늘쫑(수입),강릉,224.0,586000.0
2,20190201,조미채소류,마늘,깐마늘,강릉,1287.0,7259700.0
3,20190201,조미채소류,마늘,풋마늘,강릉,210.0,491000.0
4,20190201,조미채소류,풋고추,아삭이,강릉,62.0,244000.0


In [7]:
df_agromarket.dtypes

date                  int64
category             object
item                 object
variety              object
wholesale_market     object
quantity_kg         float64
price_won           float64
dtype: object

In [8]:
# 'date' 컬럼을 날짜 형식으로 변환
df_agromarket['date'] = pd.to_datetime(df_agromarket['date'], format='%Y%m%d')

df_agromarket.dtypes

date                datetime64[ns]
category                    object
item                        object
variety                     object
wholesale_market            object
quantity_kg                float64
price_won                  float64
dtype: object

In [9]:
# 결측치가 있는 행 제거
df_agromarket = df_agromarket.dropna()

df_agromarket.isnull().sum()

date                0
category            0
item                0
variety             0
wholesale_market    0
quantity_kg         0
price_won           0
dtype: int64

In [10]:
# 'date', 'category', 'item', 'variety', 'wholesale_market' 기준으로 중복값 확인
duplicate_count_specific_columns = df_agromarket.duplicated(
    subset=['date', 'category', 'item', 'variety', 'wholesale_market']
).sum()

duplicate_count_specific_columns

np.int64(1971410)

In [11]:
# 중복된 행 필터링
duplicates = df_agromarket[df_agromarket.duplicated(subset=['date', 'category', 'item', 'variety', 'wholesale_market'], keep=False)]
print(duplicates)


              date category item    variety wholesale_market  quantity_kg  \
37956   2021-03-26      버섯류  새송이  새송이버섯(일반)               강릉        443.0   
37957   2021-03-26      버섯류  새송이  새송이버섯(일반)               강릉        443.0   
37958   2021-03-26    조미채소류   마늘    마늘쫑(수입)               강릉        194.0   
37959   2021-03-26    조미채소류   마늘    마늘쫑(수입)               강릉        194.0   
37960   2021-03-26    조미채소류   마늘        깐마늘               강릉        193.0   
...            ...      ...  ...        ...              ...          ...   
4636289 2023-08-16      과채류   호박        단호박             인천남촌         32.0   
4636290 2023-08-16      과채류   호박        단호박             인천남촌         10.0   
4636291 2023-08-16      과채류   호박        단호박             인천남촌         30.0   
4636292 2023-08-16      과채류   호박        풋호박             인천남촌        376.0   
4636293 2023-08-16      과채류   호박        풋호박             인천남촌        300.0   

         price_won  
37956    1338480.0  
37957    1338480.0  
37958     69

In [12]:
df_agromarket.isnull().sum()

date                0
category            0
item                0
variety             0
wholesale_market    0
quantity_kg         0
price_won           0
dtype: int64

In [13]:
# 'date'를 'year', 'month', 'day'로 분리
df_agromarket['year'] = df_agromarket['date'].dt.year
df_agromarket['month'] = df_agromarket['date'].dt.month
df_agromarket['day'] = df_agromarket['date'].dt.day

# 계절(season) 컬럼 추가 함수 정의
def get_season(month):
    if month in [12, 1, 2]:
        return "겨울"
    elif month in [3, 4, 5]:
        return "봄"
    elif month in [6, 7, 8]:
        return "여름"
    else:
        return "가을"

# season 컬럼 추가
df_agromarket['season'] = df_agromarket['month'].apply(get_season)

df_agromarket[['date', 'year', 'month', 'day', 'season']].head()

,date,year,month,day,season
0,2019-02-01,2019,2,1,겨울
1,2019-02-01,2019,2,1,겨울
2,2019-02-01,2019,2,1,겨울
3,2019-02-01,2019,2,1,겨울
4,2019-02-01,2019,2,1,겨울


In [14]:
# 'variety' 컬럼 삭제
df_agromarket = df_agromarket.drop(columns=['variety'])

# 'date', 'category', 'item', 'wholesale_market' 기준으로 그룹화하여 합계와 평균 계산
grouped = df_agromarket.groupby(['date', 'category', 'item', 'wholesale_market'], as_index=False).agg({
    'quantity_kg': ['sum', 'mean'],
    'price_won': ['sum', 'mean']
})

# 멀티인덱스를 단순화하여 컬럼명 변경
grouped.columns = [
    'date', 'category', 'item', 'wholesale_market',
    'quantity_kg', 'avg_quantity_kg', 'price_won', 'avg_price_won'
]

grouped.head()


,date,category,item,wholesale_market,quantity_kg,avg_quantity_kg,price_won,avg_price_won
0,2019-01-03,과실류,감귤,강릉,20293.0,10146.500000,34098700.0,1.704935e+07
1,2019-01-03,과실류,감귤,광주각화,95470.5,15911.750000,169490200.0,2.824837e+07
2,2019-01-03,과실류,감귤,광주서부,173202.5,21650.312500,338853900.0,4.235674e+07
3,2019-01-03,과실류,감귤,구리,329985.5,29998.681818,601745213.0,5.470411e+07
4,2019-01-03,과실류,감귤,구미,24230.0,8076.666667,41805000.0,1.393500e+07


In [15]:
# 단가 계산 (0으로 나누기 방지)
grouped["unit_price_per_kg"] = grouped["price_won"] / grouped["quantity_kg"].replace(0, pd.NA)
grouped["avg_unit_price_per_kg"] = grouped["avg_price_won"] / grouped["avg_quantity_kg"].replace(0, pd.NA)

# 결측치 처리 및 단가 음수 제거
grouped = grouped.fillna(0)
grouped = grouped[(grouped["unit_price_per_kg"] >= 0) & (grouped["avg_unit_price_per_kg"] >= 0)]

grouped.head()


,date,category,item,wholesale_market,quantity_kg,avg_quantity_kg,price_won,avg_price_won,unit_price_per_kg,avg_unit_price_per_kg
0,2019-01-03,과실류,감귤,강릉,20293.0,10146.500000,34098700.0,1.704935e+07,1680.318336,1680.318336
1,2019-01-03,과실류,감귤,광주각화,95470.5,15911.750000,169490200.0,2.824837e+07,1775.314888,1775.314888
2,2019-01-03,과실류,감귤,광주서부,173202.5,21650.312500,338853900.0,4.235674e+07,1956.403054,1956.403054
3,2019-01-03,과실류,감귤,구리,329985.5,29998.681818,601745213.0,5.470411e+07,1823.550468,1823.550468
4,2019-01-03,과실류,감귤,구미,24230.0,8076.666667,41805000.0,1.393500e+07,1725.340487,1725.340487


In [16]:
# 음수값 확인
negative_values = grouped.select_dtypes(include=['number']).lt(0).sum()

# 결측치 확인
missing_values = grouped.isnull().sum()

negative_values, missing_values


(quantity_kg              0
 avg_quantity_kg          0
 price_won                0
 avg_price_won            0
 unit_price_per_kg        0
 avg_unit_price_per_kg    0
 dtype: int64,
 date                     0
 category                 0
 item                     0
 wholesale_market         0
 quantity_kg              0
 avg_quantity_kg          0
 price_won                0
 avg_price_won            0
 unit_price_per_kg        0
 avg_unit_price_per_kg    0
 dtype: int64)

In [17]:
df_agromarket.columns

Index(['date', 'category', 'item', 'wholesale_market', 'quantity_kg',
       'price_won', 'year', 'month', 'day', 'season'],
      dtype='object')

In [18]:
# 'wholesale_market'에서 앞 두 글자를 추출하여 'region' 컬럼 추가
df_agromarket['region'] = df_agromarket['wholesale_market'].str[:2]

df_agromarket.head()

,date,category,item,wholesale_market,quantity_kg,price_won,year,month,day,season,region
0,2019-02-01,버섯류,새송이,강릉,1303.0,4081500.0,2019,2,1,겨울,강릉
1,2019-02-01,조미채소류,마늘,강릉,224.0,586000.0,2019,2,1,겨울,강릉
2,2019-02-01,조미채소류,마늘,강릉,1287.0,7259700.0,2019,2,1,겨울,강릉
3,2019-02-01,조미채소류,마늘,강릉,210.0,491000.0,2019,2,1,겨울,강릉
4,2019-02-01,조미채소류,풋고추,강릉,62.0,244000.0,2019,2,1,겨울,강릉


In [19]:
df_agromarket.columns

Index(['date', 'category', 'item', 'wholesale_market', 'quantity_kg',
       'price_won', 'year', 'month', 'day', 'season', 'region'],
      dtype='object')

In [20]:
grouped.columns

Index(['date', 'category', 'item', 'wholesale_market', 'quantity_kg',
       'avg_quantity_kg', 'price_won', 'avg_price_won', 'unit_price_per_kg',
       'avg_unit_price_per_kg'],
      dtype='object')

In [21]:
# 두 데이터프레임 병합
df_final = pd.merge(
    df_agromarket,
    grouped,
    on=['date', 'category', 'item', 'wholesale_market'],
    how='left'
)

df_final.head()


,date,category,item,wholesale_market,quantity_kg_x,price_won_x,year,month,day,season,region,quantity_kg_y,avg_quantity_kg,price_won_y,avg_price_won,unit_price_per_kg,avg_unit_price_per_kg
0,2019-02-01,버섯류,새송이,강릉,1303.0,4081500.0,2019,2,1,겨울,강릉,1303.0,1303.000000,4081500.0,4081500.0,3132.386800,3132.386800
1,2019-02-01,조미채소류,마늘,강릉,224.0,586000.0,2019,2,1,겨울,강릉,1721.0,573.666667,8336700.0,2778900.0,4844.102266,4844.102266
2,2019-02-01,조미채소류,마늘,강릉,1287.0,7259700.0,2019,2,1,겨울,강릉,1721.0,573.666667,8336700.0,2778900.0,4844.102266,4844.102266
3,2019-02-01,조미채소류,마늘,강릉,210.0,491000.0,2019,2,1,겨울,강릉,1721.0,573.666667,8336700.0,2778900.0,4844.102266,4844.102266
4,2019-02-01,조미채소류,풋고추,강릉,62.0,244000.0,2019,2,1,겨울,강릉,1586.0,396.500000,10325000.0,2581250.0,6510.088272,6510.088272


In [22]:
df_final.columns

Index(['date', 'category', 'item', 'wholesale_market', 'quantity_kg_x',
       'price_won_x', 'year', 'month', 'day', 'season', 'region',
       'quantity_kg_y', 'avg_quantity_kg', 'price_won_y', 'avg_price_won',
       'unit_price_per_kg', 'avg_unit_price_per_kg'],
      dtype='object')

In [23]:
# 필요한 컬럼만 선택
df_final = pd.merge(
    df_agromarket,
    grouped,
    on=['date', 'category', 'item', 'wholesale_market'],
    how='left'
)

# 중복된 컬럼 정리
df_final = df_final.rename(columns={
    'quantity_kg_x': 'quantity_kg',
    'price_won_x': 'price_won'
}).drop(columns=['quantity_kg_y', 'price_won_y'])

# 최종 컬럼 순서 조정
df_final = df_final[[
    'date', 'year', 'month', 'day', 'season', 'region', 'wholesale_market',
    'category', 'item', 'quantity_kg', 'price_won', 'unit_price_per_kg',
    'avg_quantity_kg', 'avg_price_won', 'avg_unit_price_per_kg'
]]

df_final.head()

,date,year,month,day,season,region,wholesale_market,category,item,quantity_kg,price_won,unit_price_per_kg,avg_quantity_kg,avg_price_won,avg_unit_price_per_kg
0,2019-02-01,2019,2,1,겨울,강릉,강릉,버섯류,새송이,1303.0,4081500.0,3132.386800,1303.000000,4081500.0,3132.386800
1,2019-02-01,2019,2,1,겨울,강릉,강릉,조미채소류,마늘,224.0,586000.0,4844.102266,573.666667,2778900.0,4844.102266
2,2019-02-01,2019,2,1,겨울,강릉,강릉,조미채소류,마늘,1287.0,7259700.0,4844.102266,573.666667,2778900.0,4844.102266
3,2019-02-01,2019,2,1,겨울,강릉,강릉,조미채소류,마늘,210.0,491000.0,4844.102266,573.666667,2778900.0,4844.102266
4,2019-02-01,2019,2,1,겨울,강릉,강릉,조미채소류,풋고추,62.0,244000.0,6510.088272,396.500000,2581250.0,6510.088272


In [24]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4636256 entries, 0 to 4636255
Data columns (total 15 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   date                   datetime64[ns]
 1   year                   int32         
 2   month                  int32         
 3   day                    int32         
 4   season                 object        
 5   region                 object        
 6   wholesale_market       object        
 7   category               object        
 8   item                   object        
 9   quantity_kg            float64       
 10  price_won              float64       
 11  unit_price_per_kg      float64       
 12  avg_quantity_kg        float64       
 13  avg_price_won          float64       
 14  avg_unit_price_per_kg  float64       
dtypes: datetime64[ns](1), float64(6), int32(3), object(5)
memory usage: 477.5+ MB


In [25]:
# 숫자 데이터 소수점 둘째자리까지 처리
numeric_columns = df_final.select_dtypes(include=['float', 'int']).columns
df_final[numeric_columns] = df_final[numeric_columns].round(2)

In [ ]:
# 데이터 저장
output_path = '../data/preprocessed/preprocessed_agromarket2.csv'
df_final.to_csv(output_path, index=False)

f"데이터가 '{output_path}'에 저장되었습니다."

"데이터가 '../data/db/preprocessed_agromarket2.csv'에 저장되었습니다."